# RNNによる数値計算

In [ ]:
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 計算問題の生成(3桁+3桁の足し算問題を50000問作生成)
class CharacterTable(object):

    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')

while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))


Generating data...
Total addition questions: 50000


In [3]:
#生成された計算問題を10件分表示
for i in range(10):
    print (questions[i])
    #print (ans[i])
print (query[0])
print (expected[0])

  72+44
 53+006
  92+05
    1+0
    9+6
  0+567
    9+5
918+117
  434+5
   0+92
6
71  


In [4]:
# 問題と正解文字のベクトル化(One-Hotベクタ)
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)

for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


print (questions[0])
print (x[0])

Vectorization...
  72+44
[[ True False False False False False False False False False False False]
 [ True False False False False False False False False False False False]
 [False False False False False False False False False  True False False]
 [False False False False  True False False False False False False False]
 [False  True False False False False False False False False False False]
 [False False False False False False  True False False False False False]
 [False False False False False False  True False False False False False]]


In [5]:
#学習と検証用にデータを分割(学習用:45000, 検証用5000)

indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [6]:
# モデルの定義
# Try replacing GRU, or SimpleRNN.

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 1024
LAYERS = 1

print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 学習の実行(epoch = 80, batch_size =1024)
# 50 epochあたりから徐々に正解していく

for iteration in range(1, 80):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 3s 72us/step - loss: 2.2356 - acc: 0.2219 - val_loss: 2.0450 - val_acc: 0.2893
Q 45+314  T 359  ☒ 1   
Q 43+43   T 86   ☒ 1   
Q 60+65   T 125  ☒ 1   
Q 868+3   T 871  ☒ 1   
Q 582+7   T 589  ☒ 1   
Q 26+996  T 1022 ☒ 1   
Q 659+978 T 1637 ☒ 11  
Q 42+587  T 629  ☒ 1   
Q 292+984 T 1276 ☒ 1   
Q 701+95  T 796  ☒ 1   

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 2s 34us/step - loss: 1.9121 - acc: 0.3250 - val_loss: 1.8372 - val_acc: 0.3375
Q 313+269 T 582  ☒ 110 
Q 8+739   T 747  ☒ 117 
Q 78+742  T 820  ☒ 110 
Q 29+726  T 755  ☒ 110 
Q 20+470  T 490  ☒ 117 
Q 178+60  T 238  ☒ 110 
Q 236+49  T 285  ☒ 115 
Q 3+797   T 800  ☒ 107 
Q 967+1   T 968  ☒ 117 
Q 685+87  T 772  ☒ 110 

---------------------

45000/45000 [==============================] - 2s 33us/step - loss: 1.5263 - acc: 0.4270 - val_loss: 1.5070 - val_acc: 0.4371
Q 775+87  T 862  ☒ 852 
Q 79+375  T 454  ☒ 711 
Q 574+271 T 845  ☒ 810 
Q 788+129 T 917  ☒ 810 
Q 650+58  T 708  ☒ 653 
Q 289+80  T 369  ☒ 995 
Q 483+98  T 581  ☒ 990 
Q 673+8   T 681  ☒ 773 
Q 396+750 T 1146 ☒ 1002
Q 947+20  T 967  ☒ 901 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 2s 34us/step - loss: 1.4962 - acc: 0.4401 - val_loss: 1.4743 - val_acc: 0.4461
Q 619+0   T 619  ☒ 166 
Q 47+242  T 289  ☒ 477 
Q 265+44  T 309  ☒ 567 
Q 317+385 T 702  ☒ 808 
Q 276+147 T 423  ☒ 780 
Q 786+399 T 1185 ☒ 1201
Q 366+7   T 373  ☒ 733 
Q 80+678  T 758  ☒ 756 
Q 680+410 T 1090 ☒ 1007
Q 435+421 T 856  ☒ 818 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 2s 34us/step - loss: 1.1389 - acc: 0.5799 - val_loss: 1.1334 - val_acc: 0.5820
Q 43+537  T 580  ☑ 580 
Q 82+571  T 653  ☒ 630 
Q 81+91   T 172  ☒ 113 
Q 44+23   T 67   ☒ 47  
Q 899+6   T 905  ☒ 994 
Q 87+13   T 100  ☒ 10  
Q 409+449 T 858  ☒ 803 
Q 233+77  T 310  ☒ 300 
Q 90+364  T 454  ☒ 431 
Q 116+82  T 198  ☒ 111 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 2s 34us/step - loss: 1.1152 - acc: 0.5890 - val_loss: 1.1014 - val_acc: 0.5935
Q 474+4   T 478  ☒ 470 
Q 59+480  T 539  ☒ 506 
Q 133+51  T 184  ☒ 177 
Q 5+215   T 220  ☒ 229 
Q 485+9   T 494  ☑ 494 
Q 461+86  T 547  ☒ 554 
Q 856+65  T 921  ☑ 921 
Q 511+99  T 610  ☒ 600 
Q 125+68  T 193  ☒ 191 
Q 544+91  T 635  ☒ 623 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 2s 34us/step - loss: 0.7925 - acc: 0.7193 - val_loss: 0.7922 - val_acc: 0.7104
Q 43+508  T 551  ☒ 552 
Q 93+893  T 986  ☒ 906 
Q 749+128 T 877  ☒ 879 
Q 772+3   T 775  ☒ 774 
Q 321+4   T 325  ☑ 325 
Q 859+9   T 868  ☒ 877 
Q 133+57  T 190  ☒ 199 
Q 46+509  T 555  ☒ 569 
Q 56+116  T 172  ☒ 174 
Q 93+893  T 986  ☒ 906 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 2s 33us/step - loss: 0.7660 - acc: 0.7282 - val_loss: 0.7625 - val_acc: 0.7216
Q 941+95  T 1036 ☒ 1035
Q 43+471  T 514  ☒ 416 
Q 994+153 T 1147 ☒ 1146
Q 8+549   T 557  ☒ 566 
Q 56+41   T 97   ☒ 90  
Q 307+6   T 313  ☑ 313 
Q 490+69  T 559  ☑ 559 
Q 5+381   T 386  ☒ 383 
Q 66+667  T 733  ☒ 737 
Q 4+496   T 500  ☒ 491 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 2s 34us/step - loss: 0.3845 - acc: 0.8916 - val_loss: 0.3801 - val_acc: 0.8924
Q 279+24  T 303  ☑ 303 
Q 277+619 T 896  ☑ 896 
Q 272+6   T 278  ☑ 278 
Q 608+1   T 609  ☑ 609 
Q 74+634  T 708  ☑ 708 
Q 869+13  T 882  ☑ 882 
Q 363+835 T 1198 ☒ 1299
Q 99+857  T 956  ☒ 955 
Q 938+58  T 996  ☑ 996 
Q 599+498 T 1097 ☒ 1088

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 2s 33us/step - loss: 0.3617 - acc: 0.9029 - val_loss: 0.3569 - val_acc: 0.9035
Q 29+94   T 123  ☒ 122 
Q 322+70  T 392  ☑ 392 
Q 780+658 T 1438 ☑ 1438
Q 98+6    T 104  ☑ 104 
Q 607+995 T 1602 ☒ 1603
Q 92+92   T 184  ☑ 184 
Q 10+749  T 759  ☑ 759 
Q 785+140 T 925  ☑ 925 
Q 228+742 T 970  ☑ 970 
Q 663+250 T 913  ☑ 913 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 2s 33us/step - loss: 0.1686 - acc: 0.9726 - val_loss: 0.1824 - val_acc: 0.9631
Q 66+237  T 303  ☑ 303 
Q 8+989   T 997  ☑ 997 
Q 90+584  T 674  ☑ 674 
Q 448+1   T 449  ☑ 449 
Q 450+35  T 485  ☑ 485 
Q 44+24   T 68   ☑ 68  
Q 52+674  T 726  ☑ 726 
Q 14+925  T 939  ☑ 939 
Q 65+385  T 450  ☑ 450 
Q 927+14  T 941  ☑ 941 

--------------------------------------------------
Iteration 72
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 2s 34us/step - loss: 0.1618 - acc: 0.9735 - val_loss: 0.1695 - val_acc: 0.9675
Q 836+1   T 837  ☑ 837 
Q 95+781  T 876  ☑ 876 
Q 263+667 T 930  ☒ 920 
Q 689+505 T 1194 ☑ 1194
Q 71+15   T 86   ☑ 86  
Q 538+635 T 1173 ☑ 1173
Q 80+120  T 200  ☑ 200 
Q 458+335 T 793  ☑ 793 
Q 836+797 T 1633 ☑ 1633
Q 9+440   T 449  ☑ 449 

--------------------------------------------------
Iteration 73
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

In [ ]:
# Jupyter上でカーネルリセット&GPUメモリ解放
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()